In [ ]:
# Import python packages
import sys;
import json;

# Get JSON input parameters
y = json.loads(sys.argv[0])

# the result is a Python dictionary:
src_schema ="'" + y["src_schema"]+ "'"# Absolute name (database.schema)
target_table = "'" +y["target_table"]+ "'"
stage_name = "'" +y["stage_name"]+ "'"
stage_path_suffix = "'" +y["stage_path_suffix"]+ "'"
pattern_file_name = "'" +y["pattern_file_name"]+ "'"
file_format = "'" +y["file_format"]+ "'"
on_error = "'" + y["on_error"]+ "'"

internal_stage_asb_path = y["stage_name"] + y["stage_path_suffix"]
external_stage_abs_path = y["external_stage_root_path"] + y["stage_path_suffix"]

print("src_schema param: " + src_schema)
print("target_table param: " + target_table)
print("stage_name param: " + stage_name)
print("stage_path_suffix param: " + stage_path_suffix)
print("pattern_file_name param: " + pattern_file_name)
print("file_format param: " + file_format)
print("on_error param: " + on_error)
print("external_stage_root_path param: " + y["external_stage_root_path"])

print("Builded internal_stage_abs_path: " + internal_stage_asb_path)
print("Builded external_stage_asb_path: " + external_stage_abs_path)


In [ ]:
copy files
into {{ internal_stage_asb_path }}
from {{ external_stage_abs_path }}
PATTERN = {{ pattern_file_name }};

In [ ]:
CALL DEV_POC_VISEO_DB.bronze_layer.ingest_file_proc(
{{ src_schema }},
{{ target_table }},
{{ stage_name }},
{{ stage_path_suffix }},
{{ pattern_file_name }},
{{ file_format }},
{{ on_error }} 
);

In [ ]:
/*Check if all files are in "status" "LOADED". 
If NOT, raise an exception to fail the task*/

EXECUTE IMMEDIATE $$
DECLARE
  status_number NUMBER := (SELECT count(*) from {{ ingest_files_into_bronze_layer }} where "status" != 'LOADED') ;
  LOAD_NOT_COMPLETE_EXCEPTION EXCEPTION (-20002, 'FILES NOT LOADED COMPLETELY, CHECK MONITORING TABLE PLZ!');
BEGIN
  IF (status_number != 0) THEN
    RAISE LOAD_NOT_COMPLETE_EXCEPTION;
  END IF;
EXCEPTION
  WHEN OTHER THEN
    RAISE; -- Raise the same exception that you are handling.
END;
$$;